In [1]:
from elasticsearch import Elasticsearch

In [2]:
def setting():
    settings ={
        "settings": {
            "analysis": {
                "filter": {
                    "synonyms_filter": { # 同義語フィルターの定義
                        "type": "synonym",
                        "synonyms": [ #同義語リストの定義 (今は空の状態)
                            ]
                    }
                },
                "tokenizer": {
                    "sudachi_tokenizer": {
                        "type": "sudachi_tokenizer",
                        "discard_punctuation": True,
                        "sudachi_split":"search",
                        "resources_path": "/usr/share/elasticsearch/config/sudachi",
                        "settings_path": "/usr/share/elasticsearch/config/sudachi/sudachi_fulldict.json"
                    }
                },
                "analyzer": {
                    "sudachi_analyzer": {
                        "char_filter": [
                            "icu_normalizer", # 文字単位の正規化
                            "kuromoji_iteration_mark" # 繰り返し文字の正規化
                        ],
                        "filter": [
                            "synonyms_filter", # 同義語展開
                            # "kuromoji_baseform", # 活用語の原型化
                            # "kuromoji_part_of_speech", # 不要品詞の除去
                            # "ja_stop", #不要単語の除去
                            "kuromoji_number", # 数字の正規化
                            "kuromoji_stemmer" #長音の正規化
                        ],
                        "tokenizer": "sudachi_tokenizer",
                        "type": "custom"
                    }
                }
            }
        },
      "mappings":{
            "properties": {
                "text": {
                    "analyzer": "sudachi_analyzer",
                    "type": "text"
                    }
    }}}
    
    return settings

In [3]:
jp_index = "test"
es = Elasticsearch("elasticsearch-sudachi")
if es.indices.exists(index = jp_index):
    print(f"{jp_index}を更新します。")
    es.indices.delete(index = jp_index)

es.indices.create(index=jp_index, body=setting())

testを更新します。


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test'}

In [13]:
text = "０１０早々東京から来た"
body = {"text":text}
body = {"text":text, "analyzer":"sudachi_analyzer"}
es.indices.analyze(index=jp_index, body=body)


{'tokens': [{'token': '10',
   'start_offset': 0,
   'end_offset': 3,
   'type': 'word',
   'position': 0},
  {'token': '早早',
   'start_offset': 3,
   'end_offset': 5,
   'type': 'word',
   'position': 1},
  {'token': '東京',
   'start_offset': 5,
   'end_offset': 7,
   'type': 'word',
   'position': 2},
  {'token': 'から',
   'start_offset': 7,
   'end_offset': 9,
   'type': 'word',
   'position': 3},
  {'token': '来',
   'start_offset': 9,
   'end_offset': 10,
   'type': 'word',
   'position': 4},
  {'token': 'た',
   'start_offset': 10,
   'end_offset': 11,
   'type': 'word',
   'position': 5}]}

In [11]:
es.index(index=jp_index, body=body)
result = es.search(
            index=jp_index,
            body={"query": {"match_all": {}}}
)

In [6]:
result

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': 'ge8m8HIBILEZmxDbhZqO',
    '_score': 1.0,
    '_source': {'text': '1次リーグ●日本0（0－1、0－2）3中国○'}}]}}